<a href="https://colab.research.google.com/github/asmaayasse/Udacity-/blob/main/Smart_Absence_Camera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [9]:
!pip uninstall dlib -y
!CC=/usr/local/cuda-11.8/bin/gcc !CXX=/usr/local/cuda-1

/bin/bash: line 1: !CXX=/usr/local/cuda-1: No such file or directory


In [8]:
!pip uninstall dlib
!CC=/usr/local/cuda-CUDA_VERSION/bin/gcc !CXX=/usr/local/cuda-CUDA_VERSION/bin/g++ !pip install dlib

Found existing installation: dlib 19.24.6
Uninstalling dlib-19.24.6:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/_dlib_pybind11.cpython-310-x86_64-linux-gnu.so
    /usr/local/lib/python3.10/dist-packages/dlib-19.24.6.dist-info/*
    /usr/local/lib/python3.10/dist-packages/dlib/*
Proceed (Y/n)? y
  Successfully uninstalled dlib-19.24.6
/bin/bash: line 1: !CXX=/usr/local/cuda-CUDA_VERSION/bin/g++: No such file or directory


In [3]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split

source_dir = '/content/persons'

train_dir = '/content/data/splitted_data/train'
val_dir = '/content/data/splitted_data/val'
test_dir = '/content/data/splitted_data/test'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

for person in os.listdir(source_dir):
    person_path = os.path.join(source_dir, person)

    if os.path.isdir(person_path):
        os.makedirs(os.path.join(train_dir, person), exist_ok=True)
        os.makedirs(os.path.join(val_dir, person), exist_ok=True)
        os.makedirs(os.path.join(test_dir, person), exist_ok=True)

        images = [f for f in os.listdir(person_path) if os.path.isfile(os.path.join(person_path, f))]

        train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)
        val_images, test_images = train_test_split(test_images, test_size=0.5, random_state=42)

        for img in train_images:
            shutil.copy(os.path.join(person_path, img), os.path.join(train_dir, person, img))

        for img in val_images:
            shutil.copy(os.path.join(person_path, img), os.path.join(val_dir, person, img))

        for img in test_images:
            shutil.copy(os.path.join(person_path, img), os.path.join(test_dir, person, img))

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import os

def check_images(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    img.verify()
            except (IOError, OSError):
                os.remove(file_path)

check_images('/content/persons/data/splitted_data/train')
check_images('/content/persons/data/splitted_data/val')

train_datagen = ImageDataGenerator(rescale=1.0 / 255)
val_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    '/content/persons/data/splitted_data/train',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    '/content/persons/data/splitted_data/val',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

model = models.Sequential([
    layers.Input(shape=(128, 128, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(train_generator.class_indices), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Found 214 images belonging to 6 classes.
Found 26 images belonging to 6 classes.


In [17]:
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)

model.save('face_recognition_model.keras')

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


6/6 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - accuracy: 0.1913 - loss: 1.9357 - val_accuracy: 0.3077 - val_loss: 1.6250
Epoch 2/10
1/6 ━━━━━━━━━━━━━━━━━━━━ 4s 898ms/step - accuracy: 0.3125 - loss: 1.6152

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 580ms/step - accuracy: 0.3125 - loss: 1.6152 - val_accuracy: 0.3846 - val_loss: 1.5873
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.3991 - loss: 1.5234 - val_accuracy: 0.2692 - val_loss: 1.3749
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 672ms/step - accuracy: 0.3438 - loss: 1.4086 - val_accuracy: 0.3846 - val_loss: 1.4326
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.4756 - loss: 1.3434 - val_accuracy: 0.5385 - val_loss: 1.1458
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 591ms/step - accuracy: 0.5625 - loss: 1.2113 - val_accuracy: 0.4615 - val_loss: 1.0703
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.5472 - loss: 1.0852 - val_accuracy: 0.6538 - val_loss: 0.9321
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 611ms/step - accuracy: 0.5938 - loss: 0.9735 - val_accuracy: 0.6538 - val_loss: 0.8420
Epoch 9/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.7046 - loss: 0.7730 - val_accuracy: 0.7308 - val_loss: 0.6278
Epoch 10/10

In [32]:
# import cv2
# import numpy as np
# from tensorflow.keras.models import load_model
# from google.colab.output import eval_js
# from IPython.display import Javascript, display
# import base64
# import PIL.Image
# from io import BytesIO
# import os

# # Load the trained model
# model = load_model("face_recognition_model.keras")

# # Define the categories
# categories = {0: "Asmaa Yasser Hassan", 1: "Ahmed Nabil Soliaman", 2: "Eslam Badwway Ahmed", 3: "Maha MohamedMohamed", 4: "Marwan Sabra Ahmed"}

# # JavaScript code to capture video from the webcam
# def video_stream():
#     display(Javascript('''
#         async function captureVideo() {
#             const video = document.createElement('video');
#             video.style.display = 'none';
#             document.body.appendChild(video);
#             const stream = await navigator.mediaDevices.getUserMedia({video: true});
#             video.srcObject = stream;
#             await video.play();

#             const canvas = document.createElement('canvas');
#             document.body.appendChild(canvas);
#             canvas.width = video.videoWidth;
#             canvas.height = video.videoHeight;

#             const context = canvas.getContext('2d');
#             return new Promise((resolve) => {
#                 const interval = setInterval(() => {
#                     context.drawImage(video, 0, 0, canvas.width, canvas.height);
#                     const dataUrl = canvas.toDataURL('image/jpeg');
#                     google.colab.kernel.invokeFunction('notebook.requestFrame', { data: dataUrl }, {});
#                 }, 100);
#             });
#         }
#         captureVideo();
#     '''))

# def decode_base64_image(data_url):
#     """Decodes a base64 image and converts it into a numpy array."""
#     data = base64.b64decode(data_url.split(',')[1])
#     image = PIL.Image.open(BytesIO(data))
#     return np.array(image)

# # Start the video stream
# video_stream()

# # Process the video frames
# def process_frames():
#     while True:
#         # Wrap arguments in an array within a dictionary
#         frame_data = eval_js("google.colab.kernel.invokeFunction('notebook.requestFrame', [{}], {}).then(r => r.data)")
#         if frame_data:
#             frame = decode_base64_image(frame_data)

#             # Resize the frame for the model
#             img_resized = cv2.resize(frame, (128, 128))
#             img_resized = np.expand_dims(img_resized, axis=0) / 255.0

#             # Make predictions
#             predictions = model.predict(img_resized)
#             class_idx = np.argmax(predictions[0])
#             class_name = categories[class_idx]

#             # Display the prediction on the frame
#             cv2.putText(frame, f"Person: {class_name}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
#             cv2.imshow("Camera", frame)

#             # Exit on pressing 'q'
#             if cv2.waitKey(1) & 0xFF == ord('q'):
#                 break

#     cv2.destroyAllWindows()

# # Run the frame processing
# process_frames()

In [44]:
# !apt update
# !apt install -y --no-install-recommends cuda-drivers
# !pip install opencv-python

import cv2
import numpy as np
from google.colab.output import eval_js
from IPython.display import Javascript, display
import base64
import PIL.Image
from io import BytesIO
import os

# Load the known face images and their names
known_face_names = []
known_face_encodings = []

for file in os.listdir("known_faces"):
    if file.endswith((".jpg", ".jpeg", ".png")):
        image_path = os.path.join("known_faces", file)
        name = os.path.splitext(file)[0]

        image = face_recognition.load_image_file(image_path)
        face_encoding = face_recognition.face_encodings(image)[0]

        known_face_names.append(name)
        known_face_encodings.append(face_encoding)

# JavaScript code to capture video from the webcam
def video_stream():
    js = Javascript('''
        async function captureVideo() {
            const video = document.createElement('video');
            video.style.display = 'none';
            document.body.appendChild(video);
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await video.play();

            const canvas = document.createElement('canvas');
            document.body.appendChild(canvas);
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;

            const context = canvas.getContext('2d');
            return new Promise((resolve) => {
                const interval = setInterval(() => {
                    context.drawImage(video, 0, 0, canvas.width, canvas.height);
                    const dataUrl = canvas.toDataURL('image/jpeg');
                    google.colab.kernel.invokeFunction('notebook.requestFrame', { data: dataUrl }, {});
                }, 100);
            });
        }
        captureVideo();
    ''')
    display(js)

# Function to decode base64 image to numpy array
def decode_base64_image(data_url):
    """Decodes a base64 image and converts it into a numpy array."""
    data = base64.b64decode(data_url.split(',')[1])
    image = PIL.Image.open(BytesIO(data))
    return np.array(image)

# Process the video frames
def process_frames():
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    while True:
        # Get frame data from JavaScript
        frame_data = eval_js("google.colab.kernel.invokeFunction('notebook.requestFrame', [{}], {}).then(r => r.data)")
        if frame_data:
            # Decode the frame data
            frame = decode_base64_image(frame_data)

            # Convert the frame to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Detect faces in the frame
            faces = face_cascade.detectMultiScale(gray, 1.1, 4)

            # Loop through each detected face
            for (x, y, w, h) in faces:
                # Draw a rectangle around the face
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

                # Get the face encoding for the detected face
                face_encoding = face_recognition.face_encodings(frame, [(y, x + w, y + h, x)])[0]  # Adjust coordinates

                # Compare the face encoding with known face encodings
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

                name = "Unknown"
                if True in matches:
                    first_match_index = matches.index(True)
                    name = known_face_names[first_match_index]

                # Display the name of the recognized person
                cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

            # Display the resulting image
            cv2.imshow('Video', frame)

            # Hit 'q' on the keyboard to quit!
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cv2.destroyAllWindows()

# Start the video stream
video_stream()

# Run the frame processing
process_frames()

FileNotFoundError: [Errno 2] No such file or directory: 'known_faces'

In [10]:
# !pip install face_recognition
import face_recognition
import cv2
import numpy as np
from google.colab.output import eval_js
from IPython.display import Javascript, display
import base64
import PIL.Image
from io import BytesIO
import os

# Create the known_faces directory if it doesn't exist
os.makedirs("known_faces", exist_ok=True)

# Load the known face images and their names
known_face_names = []
known_face_encodings = []

for file in os.listdir("known_faces"):
    if file.endswith((".jpg", ".jpeg", ".png")):
        image_path = os.path.join("known_faces", file)
        name = os.path.splitext(file)[0]

        image = face_recognition.load_image_file(image_path)
        face_encoding = face_recognition.face_encodings(image)[0]

        known_face_names.append(name)
        known_face_encodings.append(face_encoding)

# JavaScript code to capture video from the webcam
def video_stream():
    js = Javascript('''
        async function captureVideo() {
            const video = document.createElement('video');
            video.style.display = 'none';
            document.body.appendChild(video);
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await video.play();

            const canvas = document.createElement('canvas');
            document.body.appendChild(canvas);
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;

            const context = canvas.getContext('2d');
            return new Promise((resolve) => {
                const interval = setInterval(() => {
                    context.drawImage(video, 0, 0, canvas.width, canvas.height);
                    const dataUrl = canvas.toDataURL('image/jpeg');
                    google.colab.kernel.invokeFunction('notebook.requestFrame', { data: dataUrl }, {});
                }, 100);
            });
        }
        captureVideo();
    ''')
    display(js)

# Function to decode base64 image to numpy array
def decode_base64_image(data_url):
    """Decodes a base64 image and converts it into a numpy array."""
    data = base64.b64decode(data_url.split(',')[1])
    image = PIL.Image.open(BytesIO(data))
    return np.array(image)

# Process the video frames
def process_frames():
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    while True:
        # Get frame data from JavaScript
        frame_data = eval_js("google.colab.kernel.invokeFunction('notebook.requestFrame', [{}], {}).then(r => r.data)")
        if frame_data:
            # Decode the frame data
            frame = decode_base64_image(frame_data)

            # Convert the frame to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Detect faces in the frame
            faces = face_cascade.detectMultiScale(gray, 1.1, 4)

            # Loop through each detected face
            for (x, y, w, h) in faces:
                # Draw a rectangle around the face
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

                # Get the face encoding for the detected face
                face_encoding = face_recognition.face_encodings(frame, [(y, x + w, y + h, x)])[0]

                # Compare the face encoding with known face encodings
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

                name = "Unknown"
                if True in matches:
                    first_match_index = matches.index(True)
                    name = known_face_names[first_match_index]

                # Display the name of the recognized person
                cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

            # Display the resulting image
            cv2.imshow('Video', frame)

            # Hit 'q' on the keyboard to quit!
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cv2.destroyAllWindows()

# Start the video stream
video_stream()

# Run the frame processing
process_frames()

RuntimeError: Error while calling cudaGetDevice(&the_device_id) in file /tmp/pip-install-gjenhzke/dlib_ef593ae5bb7748eca8fc570046e25753/dlib/cuda/gpu_data.cpp:204. code: 100, reason: no CUDA-capable device is detected

In [5]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [3]:
!pip install --upgrade --force-reinstall dlib --install-option="--no" --install-option="DLIB_USE_CUDA"


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --install-option


In [46]:
!apt update
!apt install -y --no-install-recommends cuda-drivers
!pip install opencv-python

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,197 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,632 kB]
Fetched 4,225 kB in 2s (1,964 kB/s)
Reading pack

In [1]:
!pip install --upgrade dlib
